In [1]:
import ee
import unetgee
from datetime import datetime
from dateutil.relativedelta import relativedelta
import random

2024-04-10 06:02:02.112144: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-10 06:02:02.136110: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-10 06:02:03.930903: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-10 06:02:03.935095: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bu

In [2]:
ee.Authenticate(force=True)
ee.Initialize()


Successfully saved authorization token.


In [8]:
hybas_gt100_id_list = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroBASINS_lev02/hybas_lev02_v1c_merged_lake_count_added_gt100").aggregate_array("HYBAS_ID").getInfo()
print(hybas_gt100_id_list)

[1020000010, 8020000010, 4020000010, 5020000010, 2020000010, 9020000010, 7020000010, 6020000010, 3020000010, 1020011530, 8020008900, 4020006940, 5020015660, 2020003440, 7020014250, 6020006540, 3020003790, 1020018110, 8020010700, 4020015090, 5020037270, 2020018240, 7020021430, 6020008320, 3020005240, 1020021940, 8020020760, 4020024190, 5020049720, 2020024230, 7020024600, 6020014330, 3020008670, 1020027430, 8020022890, 4020034510, 2020033490, 7020038340, 6020017370, 3020009320, 1020034170, 8020032840, 2020041390, 7020046750, 6020021870, 1020035180, 8020044560, 5020082270, 2020057170, 7020047840, 6020029280, 2020065840, 7020065090, 2020071190, 4020050210, 3020024310, 4020050220, 1020040190, 4020050290, 4020050470]


In [3]:
def read_wkt(file_path):
    # Step 1: Read the WKT string from the file
    with open(file_path, 'r') as file:
        wkt_string = file.read()
    return wkt_string

In [10]:
to_be_processed_hybas_gt100_id_list = [
    3020024310, 4020050220, 1020040190, 4020050290, 4020050470
]
sub_numbers = 0
start_date = "2001-01-01"
end_date = "2022-01-01"
date_format = "%Y-%m-%d"
start_date = datetime.strptime(start_date, date_format)
end_date = datetime.strptime(end_date, date_format)
sample_types = [['training', 'eval', 'test'][(i+year-2001)%3] for year in range(2001, 2024) for i in range(12)]
if sub_numbers > 0:
    print("sub_numbers: ", sub_numbers)
    sample_sizes = [
        [1000, 0, 0],
        [0, 250, 0],
        [0, 0, 250]
    ]
else:
    print("No sub_numbers")
    sample_sizes = [
        [4000, 0, 0],
        [0, 1000, 0],
        [0, 0, 1000]
    ]
#fill_date_list = [(datetime.strptime("2015-08-01", date_format) + relativedelta(months=i)).strftime(date_format) for i in range(0, 12*14)]
fill_date_list = []
print(fill_date_list)

projection_type = 'Lambert_Azimuthal_Equal_Area'
projection_scale = 30
projection_wkt_path = './projection_wkt/{projection_type}/{hybas_id}.wkt'

for hybas_id in to_be_processed_hybas_gt100_id_list:
    if sub_numbers > 0:
        for sub in range(sub_numbers):
            current_date = start_date
            print("processing hybas_id: ", hybas_id)
            current_projection_wkt_path = projection_wkt_path.format(projection_type=projection_type, projection_scale=projection_scale, hybas_id=hybas_id)
            current_projection_wkt = read_wkt(current_projection_wkt_path)
            current_region = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES_per_basin_dissolved_simplified/hylak_buffered_bounds_dissolved_simplified_1000m_{}_sub{}".format(hybas_id, sub+1)).geometry(1)
            mon_count = 0
            while current_date < end_date:
                if fill_date_list and current_date.strftime(date_format) not in fill_date_list:
                    current_date += relativedelta(months=1)
                    mon_count += 1
                    continue
                current_type = sample_types[mon_count]
                current_sizes = sample_sizes[['training', 'eval', 'test'].index(current_type)]
                current_end_date = current_date + relativedelta(months=1)
                random.seed(current_date.year)
                starting_seed = random.randint(0, 100000)
                unetgee.unet_sample_generation_time_period(
                    region=current_region,
                    start_time=current_date.strftime(date_format),
                    end_time=current_end_date.strftime(date_format),
                    generate_samples=True,
                    train_size=current_sizes[0], eval_size=current_sizes[1], test_size=current_sizes[2],
                    train_shard_num=5, eval_shard_num=5, test_shard_num=5,
                    shard_size=50,
                    training_base=f"training_{hybas_id}_sub{sub+1}_{current_date.strftime(date_format)}_{current_end_date.strftime(date_format)}",
                    eval_base=f"eval_{hybas_id}_sub{sub+1}_{current_date.strftime(date_format)}_{current_end_date.strftime(date_format)}",
                    test_base=f"test_{hybas_id}_sub{sub+1}_{current_date.strftime(date_format)}_{current_end_date.strftime(date_format)}",
                    starting_seed=starting_seed,
                    export_to='drive',
                    drive_folder=f'10bands_LAEA_samples_{hybas_id}',
                    input_bands=['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2', 'GSW_Occurrence', 'GSW_Recurrence'],
                    custom_projection_wkt=current_projection_wkt,
                    custom_scale=projection_scale,
                    mode='joint_neighborhood_custom_projection'
                )
                print(f"Generate {current_type} samples for basin {hybas_id}_sub{sub+1} from ", current_date.strftime(date_format), " to ", current_end_date.strftime(date_format))
                current_date += relativedelta(months=1)
                mon_count += 1
    
    else:
        current_date = start_date
        print("processing hybas_id: ", hybas_id)
        current_projection_wkt_path = projection_wkt_path.format(projection_type=projection_type, projection_scale=projection_scale, hybas_id=hybas_id)
        current_projection_wkt = read_wkt(current_projection_wkt_path)
        current_region = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES_per_basin_dissolved_simplified/hylak_buffered_bounds_dissolved_simplified_1000m_{}".format(hybas_id)).geometry(1)
        mon_count = 0
        while current_date < end_date:
            if fill_date_list and current_date.strftime(date_format) not in fill_date_list:
                current_date += relativedelta(months=1)
                mon_count += 1
                continue
            current_type = sample_types[mon_count]
            current_sizes = sample_sizes[['training', 'eval', 'test'].index(current_type)]
            current_end_date = current_date + relativedelta(months=1)
            random.seed(current_date.year)
            starting_seed = random.randint(0, 100000)
            unetgee.unet_sample_generation_time_period(
                region=current_region,
                start_time=current_date.strftime(date_format),
                end_time=current_end_date.strftime(date_format),
                generate_samples=True,
                train_size=current_sizes[0], eval_size=current_sizes[1], test_size=current_sizes[2],
                shard_size=50,
                training_base=f"training_{hybas_id}_{current_date.strftime(date_format)}_{current_end_date.strftime(date_format)}",
                eval_base=f"eval_{hybas_id}_{current_date.strftime(date_format)}_{current_end_date.strftime(date_format)}",
                test_base=f"test_{hybas_id}_{current_date.strftime(date_format)}_{current_end_date.strftime(date_format)}",
                starting_seed=starting_seed,
                export_to='drive',
                drive_folder=f'10bands_LAEA_samples_{hybas_id}',
                input_bands=['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2', 'GSW_Occurrence', 'GSW_Recurrence'],
                custom_projection_wkt=current_projection_wkt,
                custom_scale=projection_scale,
                mode='joint_neighborhood_custom_projection'
            )
            print(f"Generate {current_type} samples for basin {hybas_id} from ", current_date.strftime(date_format), " to ", current_end_date.strftime(date_format))
            current_date += relativedelta(months=1)
            mon_count += 1

No sub_numbers
[]
processing hybas_id:  3020024310
using regular filter for gsw, start date: 2001-01-01
Generate training samples for basin 3020024310 from  2001-01-01  to  2001-02-01
using regular filter for gsw, start date: 2001-02-01
Generate eval samples for basin 3020024310 from  2001-02-01  to  2001-03-01
using regular filter for gsw, start date: 2001-03-01
Generate test samples for basin 3020024310 from  2001-03-01  to  2001-04-01
using regular filter for gsw, start date: 2001-04-01
Generate training samples for basin 3020024310 from  2001-04-01  to  2001-05-01
using regular filter for gsw, start date: 2001-05-01
Generate eval samples for basin 3020024310 from  2001-05-01  to  2001-06-01
using regular filter for gsw, start date: 2001-06-01
Generate test samples for basin 3020024310 from  2001-06-01  to  2001-07-01
using regular filter for gsw, start date: 2001-07-01
Generate training samples for basin 3020024310 from  2001-07-01  to  2001-08-01
using regular filter for gsw, star